## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")

2022-11-13 11:01:32.546612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=["EIN","NAME"])


In [3]:
# Generate our categorical variable lists
cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [4]:
# Convert categorical data to numeric with `pd.get_dummies`
new_df = pd.get_dummies(data=df, columns=['APPLICATION_TYPE','INCOME_AMT', 'AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS'])
new_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [5]:
# Split our preprocessed data into our features and target arrays
# Remove IS_SUCCESSFUL target from features data
y = new_df.IS_SUCCESSFUL.values
X = new_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [7]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1170      
                                                                 
 dense_1 (Dense)             (None, 12)                132       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 1,315
Trainable params: 1,315
Non-trainable params: 0


2022-11-13 11:01:48.139177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


_________________________________________________________________


In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 6ms/step - loss: 0.6099 - accuracy: 0.6927
Epoch 2/100
804/804 [==============================] - 9s 12ms/step - loss: 0.5585 - accuracy: 0.7310
Epoch 3/100
804/804 [==============================] - 6s 8ms/step - loss: 0.5512 - accuracy: 0.7331
Epoch 4/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5490 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5473 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5461 - accuracy: 0.7335
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5449 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5448 - accuracy: 0.7347
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5438 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5437 - accu

804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.7394
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5355 - accuracy: 0.7391
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7385
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7401
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7393
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7390
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.7398
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7390
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.7390
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5351 - accuracy: 

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 3s - loss: 0.5577 - accuracy: 0.7251 - 3s/epoch - 10ms/step
Loss: 0.5577426552772522, Accuracy: 0.7251312136650085


In [17]:
# Export our model to HDF5 file
from tensorflow.keras.models import load_model
model=nn
model.save('opt_model2.HDF5')

INFO:tensorflow:Assets written to: opt_model2.HDF5/assets
